#### Importación de módulos

In [20]:
import pyspark.sql.functions as f_
from pyspark.sql.functions import current_timestamp
from itertools import chain
from pyspark.sql.window import Window
import sys
sys.path.append('/lakehouse/default/Files/GENERALES')
import f_amr

StatementMeta(, 08303616-ced7-4858-b382-9c890d31be05, 22, Finished, Available, Finished)

- ##### Construcción de Funnel

###### Selección de columnas iniciales

In [21]:
resumen_funnel_detalle = spark.read.parquet('abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/5182b19a-e57c-4163-83a9-b0b90997f691/Files/ANALYTIKUS/03.RESUMEN_FUNNEL_DETALLE_V2.parquet')
resumen_funnel_detalle = resumen_funnel_detalle.withColumn('FECHA', f_.to_date('FECHA', "yyyy-MM-dd"))
resumen_funnel_detalle = f_amr.cast_cols(resumen_funnel_detalle, {'LEADS':'int', 'ASISTENCIAS':'int', 'CITAS':'int', 'CONTACTO_EFECTIVO':'int', 'INSCRITOS_NETOS_REAL':'int', 'INSCRITOS_BRUTOS_REAL':'int', 'INSCRITOS_BRUTOS_ONL_REAL':'int', 'INSCRITOS_NETOS_ONL_REAL':'int'})
resumen_funnel_detalle = resumen_funnel_detalle.filter(f_.col('AÑO_COMERCIAL') == '2025')
for i in ['BLN', 'CAMPUS_BI', 'FUENTE', 'FUENTE_AGRUPADA', 'LINEA_AGRUPADA', 'LINEA_NEGOCIO_BI', 'MERCADO', 'MODALIDAD_AGRUPADA', 'MODALIDAD_BI', 'PROGRAMA_BI', 'TEAM_PRODUCTOR', 'TIPO_INGRESO_BI', 'VERTICAL_BI']:
    resumen_funnel_detalle = resumen_funnel_detalle.withColumn(i, f_.upper(f_.col(i)))

seleccion_4 = \
{'AÑO_COMERCIAL':'AÑO_COMERCIAL',
'ASISTENCIAS':'ASISTENCIAS',
'BLN':'BLN',
'CAMPUS_BI':'CAMPUS_BI',
'CICLO':'CICLO_BI',
'CITAS':'CITAS',
'CONTACTO_EFECTIVO':'CE',
'FECHA':'FECHA',
'FUENTE':'FUENTE',
'FUENTE_AGRUPADA':'FUENTE_AGRUPADA',
'INSCRITOS_BRUTOS_ONL_REAL':'INSCRITOS_BRUTOS_ONL_REAL',
'INSCRITOS_BRUTOS_REAL':'INSCRITOS_BRUTOS_REAL',
'INSCRITOS_NETOS_ONL_REAL':'INSCRITOS_NETOS_ONL_REAL',
'INSCRITOS_NETOS_REAL':'INSCRITOS_NETOS_REAL',
'LEADS':'LEADS',
'LINEA_AGRUPADA':'LINEA_AGRUPADA',
'LINEA_NEGOCIO_BI':'LINEA_NEGOCIO_BI',
'MERCADO':'MERCADO',
'MODALIDAD_AGRUPADA':'MODALIDAD_AGRUPADA',
'MODALIDAD_BI':'MODALIDAD_BI',
'NO_DE_PERSONA':'NO_DE_PERSONA',
'PROGRAMA_BI':'PROGRAMA_BI',
'TEAM_PRODUCTOR':'TEAM_PRODUCTOR',
'TIPO_BASE_BI':'TIPO_BASE_BI',
'TIPO_INGRESO_BI':'TIPO_DE_INGRESO_BI',
'VERTICAL_BI':'VERTICAL_BI'}

resumen_funnel_detalle = f_amr.select_cols(resumen_funnel_detalle, seleccion_4)
resumen_funnel_detalle = resumen_funnel_detalle.withColumn('CAMPUS_BI',f_.when(f_.col('CAMPUS_BI') == 'ONLINE', "EN LINEA").when(f_.col('CAMPUS_BI') == 'GUADALAJARA', "GUADALAJARA SUR").when(f_.col('CAMPUS_BI') == "SAN LUIS", "SAN LUIS POTOSI").otherwise(f_.col('CAMPUS_BI')))
resumen_funnel_detalle = resumen_funnel_detalle.withColumn('INDICE', f_.row_number().over(Window.orderBy('NO_DE_PERSONA', 'FECHA')))


StatementMeta(, 08303616-ced7-4858-b382-9c890d31be05, 23, Finished, Available, Finished)

###### Adición y corrección de columnas básicas KPI

- Catálogos

In [22]:
#Fuente
funnel_uvm_a = spark.read.parquet('abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/5182b19a-e57c-4163-83a9-b0b90997f691/Files/ANALYTIKUS/FUNNEL_UVM_A.parquet')
funnel_uvm_a = funnel_uvm_a.withColumn('FECHA', f_.to_date('FECHA', "yyyy-MM-dd"))
#Limpieza de columnas
seleccion_2 = \
{'CAMPUS_BI':'CAMPUS_BI',
'REGION_BI':'REGION_BI',
'TIPO_RECICLADO':'TIPO_RECICLADO',
'TRAMITE':'TRAMITE',
'CICLO_BI':'CICLO_BI',
'ESTATUS_CAMPUS':'ESTATUS_CAMPUS',
'ETAPA':'ETAPA',
'FECHA':'FECHA',
'NO_DE_PERSONA':'NO_DE_PERSONA',
'RECICLADO':'RECICLADO',
'TIPO_TRAMITE':'TIPO_TRAMITE',
'ZONA_BI':'ZONA_BI'}

funnel_uvm_a = f_amr.select_cols(funnel_uvm_a, seleccion_2)
funnel_uvm_a = f_amr.uniques(funnel_uvm_a, ['NO_DE_PERSONA', 'CICLO_BI', 'ETAPA'], [('NO_DE_PERSONA', 'asc'),('FECHA', 'desc')])
funnel_uvm_a = funnel_uvm_a.drop('ETAPA', 'FECHA')
cat_tramite = funnel_uvm_a.select('NO_DE_PERSONA', 'CICLO_BI', 'RECICLADO', 'TIPO_TRAMITE', 'TRAMITE').distinct()
cat_campus = funnel_uvm_a.select('CAMPUS_BI', 'REGION_BI', 'ESTATUS_CAMPUS', 'ZONA_BI').distinct()

StatementMeta(, 08303616-ced7-4858-b382-9c890d31be05, 24, Finished, Available, Finished)

- Ajuste columnas KPI

In [23]:
# Adición selectiva
match_tr_1 = resumen_funnel_detalle.join(cat_tramite,['NO_DE_PERSONA','CICLO_BI'], 'inner').dropDuplicates(['INDICE'])
no_match_tr_left_1 = resumen_funnel_detalle.join(cat_tramite,['NO_DE_PERSONA','CICLO_BI'], 'left_anti')
no_match_tr_right_1 = cat_tramite.join(resumen_funnel_detalle,['NO_DE_PERSONA','CICLO_BI'], 'left_anti')

match_tr_2 = no_match_tr_left_1.join(no_match_tr_right_1.drop('CICLO_BI'), 'NO_DE_PERSONA', 'inner').dropDuplicates(['INDICE'])
no_match_tr_left_2 = no_match_tr_left_1.join(no_match_tr_right_1, 'NO_DE_PERSONA', 'left_anti')

resumen_funnel_detalle = match_tr_1.unionByName(match_tr_2, allowMissingColumns=True).unionByName(no_match_tr_left_2, allowMissingColumns=True)
resumen_funnel_detalle = resumen_funnel_detalle.join(cat_campus, 'CAMPUS_BI', 'left').dropDuplicates(['INDICE'])

# Normalización post uniones
resumen_funnel_detalle = resumen_funnel_detalle.withColumn('CICLO_LAUREATE',\
f_.when(f_.col("CICLO_BI").endswith('1'), 'C1')\
.when(f_.col("CICLO_BI").endswith('2'), 'C2')\
.when(f_.col("CICLO_BI").endswith('3'), 'C3').otherwise(None))

resumen_funnel_detalle = resumen_funnel_detalle.withColumn("CICLO", f_.concat_ws("-", f_.substring(f_.col("CICLO_BI"), 3, 2),  f_.col("CICLO_BI").endswith('1')))

# Adición de Semana
calendario = spark.read.option('header', True).option('inferSchema', True).csv('abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/5182b19a-e57c-4163-83a9-b0b90997f691/Files/ANALYTIKUS/CALENDARIO_2025.csv')
calendario = calendario.withColumn('FECHA', f_.to_date('FECHA', 'dd/MM/yyyy'))
calendario = f_amr.cast_cols(calendario, {'SEMANA':'int'})

resumen_funnel_detalle = resumen_funnel_detalle.join(calendario.select('FECHA', 'SEMANA'), on='FECHA', how='inner').drop('INDICE')

StatementMeta(, 08303616-ced7-4858-b382-9c890d31be05, 25, Finished, Available, Finished)

- Etapas

In [29]:
curva_segmentos = spark.read.parquet('abfss://8c61db5f-46e1-4c72-8d6f-648bf05cacac@onelake.dfs.fabric.microsoft.com/0d2e9829-8a43-4592-a9d7-db37f7032a1c/Files/BASES_EN_PROCESO/CURVA_SEGMENTOS_UVM')

# Curva segmentos agrupada
seleccion_5 = \
['CICLO',
'ESTATUS_MARCACION',
'general_risk_bi',
'general_risk_crm',
'general_risk_prueba',
'general_risk_unicos',
'CALL_SCORE',
'LEAD_SCORE',
'SCORE_UNICOS',
'SEGMENTO_DESGLOSE',
'SEGMENTO_GENERAL',
'SEGMENTO_VIGENTE',
'NO_DE_PERSONA'
]
curva_segmentos_agrupada = curva_segmentos.select(seleccion_5)
curva_segmentos_agrupada = curva_segmentos_agrupada.distinct()

StatementMeta(, 08303616-ced7-4858-b382-9c890d31be05, 31, Finished, Available, Finished)

In [30]:
# Contacto efectivo
ce = resumen_funnel_detalle.filter(f_.col('CE') == 1).withColumn('ETAPA', f_.lit('CE'))
ce = ce.withColumn('INDICE', f_.row_number().over(Window.orderBy('NO_DE_PERSONA', 'CICLO')))

match_ce_curvseg_1 = ce.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'inner').dropDuplicates(['INDICE'])
no_match_ce_curvseg_1_left = ce.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'left_anti')
no_match_ce_curvseg_1_right = curva_segmentos_agrupada.join(ce, ['NO_DE_PERSONA','CICLO'], 'left_anti')
match_ce_curvseg_2 = no_match_ce_curvseg_1_left.join(no_match_ce_curvseg_1_right.drop('CICLO'), 'NO_DE_PERSONA', 'inner').dropDuplicates(['INDICE'])

ce = match_ce_curvseg_1.unionByName(match_ce_curvseg_2, allowMissingColumns=True)

# Registros
leads = resumen_funnel_detalle.filter(f_.col('LEADS') == 1).withColumn('ETAPA', f_.lit('LEADS'))
leads = leads.withColumn('INDICE', f_.row_number().over(Window.orderBy('NO_DE_PERSONA', 'CICLO')))

match_leads_curvseg_1 = leads.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'inner').dropDuplicates(['INDICE'])
no_match_leads_curvseg_1_left = leads.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'left_anti')
no_match_leads_curvseg_1_right = curva_segmentos_agrupada.join(leads, ['NO_DE_PERSONA','CICLO'], 'left_anti')
match_leads_curvseg_2 = no_match_leads_curvseg_1_left.join(no_match_leads_curvseg_1_right.drop('CICLO'), 'NO_DE_PERSONA', 'inner').dropDuplicates(['INDICE'])

leads = match_leads_curvseg_1.unionByName(match_leads_curvseg_2, allowMissingColumns=True)

# Citas
citas = resumen_funnel_detalle.filter(f_.col('CITAS') == 1).withColumn('ETAPA', f_.lit('CITAS'))
citas = citas.withColumn('INDICE', f_.row_number().over(Window.orderBy('NO_DE_PERSONA', 'CICLO')))

match_citas_curvseg_1 = citas.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'inner').dropDuplicates(['INDICE'])
no_match_citas_curvseg_1_left = citas.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'left_anti')
no_match_citas_curvseg_1_right = curva_segmentos_agrupada.join(citas, ['NO_DE_PERSONA','CICLO'], 'left_anti')
match_citas_curvseg_2 = no_match_citas_curvseg_1_left.join(no_match_citas_curvseg_1_right.drop('CICLO'), 'NO_DE_PERSONA', 'inner').dropDuplicates(['INDICE'])

citas = match_citas_curvseg_1.unionByName(match_citas_curvseg_2, allowMissingColumns=True)

# Asistencias
asistencias = resumen_funnel_detalle.filter(f_.col('ASISTENCIAS') == 1).withColumn('ETAPA', f_.lit('ASISTENCIAS'))
asistencias = asistencias.withColumn('INDICE', f_.row_number().over(Window.orderBy('NO_DE_PERSONA', 'CICLO')))

match_asistencias_curvseg_1 = asistencias.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'inner').dropDuplicates(['INDICE'])
no_match_asistencias_curvseg_1_left = asistencias.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'left_anti')
no_match_asistencias_curvseg_1_right = curva_segmentos_agrupada.join(asistencias, ['NO_DE_PERSONA','CICLO'], 'left_anti')
match_asistencias_curvseg_2 = no_match_asistencias_curvseg_1_left.join(no_match_asistencias_curvseg_1_right.drop('CICLO'), 'NO_DE_PERSONA', 'inner').dropDuplicates(['INDICE'])

asistencias = match_asistencias_curvseg_1.unionByName(match_asistencias_curvseg_2, allowMissingColumns=True)

# Inscritos
inscritos = resumen_funnel_detalle.filter(f_.col('INSCRITOS_BRUTOS_ONL_REAL') == 1).withColumn('ETAPA', f_.lit('INSCRITOS'))
inscritos = inscritos.withColumn('INDICE', f_.row_number().over(Window.orderBy('NO_DE_PERSONA', 'CICLO')))

match_inscritos_curvseg_1 = inscritos.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'inner').dropDuplicates(['INDICE'])
no_match_inscritos_curvseg_1_left = inscritos.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'left_anti')
no_match_inscritos_curvseg_1_right = curva_segmentos_agrupada.join(inscritos, ['NO_DE_PERSONA','CICLO'], 'left_anti')
match_inscritos_curvseg_2 = no_match_inscritos_curvseg_1_left.join(no_match_inscritos_curvseg_1_right.drop('CICLO'), 'NO_DE_PERSONA', 'inner').dropDuplicates(['INDICE'])

inscritos = match_inscritos_curvseg_1.unionByName(match_inscritos_curvseg_2, allowMissingColumns=True)

# Inscritos netos
inscritos_n = resumen_funnel_detalle.filter(f_.col('INSCRITOS_NETOS_ONL_REAL').isNotNull()).withColumn('ETAPA', f_.lit('INSCRITOS NETOS'))
inscritos_n = inscritos_n.withColumn('INDICE', f_.row_number().over(Window.orderBy('NO_DE_PERSONA', 'CICLO')))

match_inscritosn_curvseg_1 = inscritos_n.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'inner').dropDuplicates(['INDICE'])
no_match_inscritosn_curvseg_1_left = inscritos_n.join(curva_segmentos_agrupada, ['NO_DE_PERSONA','CICLO'], 'left_anti')
no_match_inscritosn_curvseg_1_right = curva_segmentos_agrupada.join(inscritos_n, ['NO_DE_PERSONA','CICLO'], 'left_anti')
match_inscritosn_curvseg_2 = no_match_inscritosn_curvseg_1_left.join(no_match_inscritosn_curvseg_1_right.drop('CICLO'), 'NO_DE_PERSONA', 'inner').dropDuplicates(['INDICE'])

inscritos_n = match_inscritosn_curvseg_1.unionByName(match_inscritosn_curvseg_2, allowMissingColumns=True)



#Funnel
apilado_funnel = ce.unionByName(leads, allowMissingColumns=True)\
                   .unionByName(citas, allowMissingColumns=True)\
                   .unionByName(asistencias, allowMissingColumns=True)\
                   .unionByName(inscritos, allowMissingColumns=True)\
                   .unionByName(inscritos_n, allowMissingColumns=True)
apilado_funnel = apilado_funnel.withColumn('DB_DISPONIBLE', f_.lit(0))

curva_intentos_score = curva_segmentos.unionByName(apilado_funnel, allowMissingColumns=True).drop(*['INDICE', 'CICLO_LAUREATE'])

StatementMeta(, 08303616-ced7-4858-b382-9c890d31be05, 32, Finished, Available, Finished)

- Normalización de columnas

In [32]:
# Creación de columnas de identificación y corrección de filtros
ajustes_cols1 = \
{
 'Id_Persona_Ciclo': f_.concat_ws( '', 'NO_DE_PERSONA', 'CICLO'),
 'FECHA_CORTE': f_.expr("current_timestamp() - INTERVAL 1 DAY"),
 'CORTE': f_.when(f_.col("FECHA") <= f_.col("FECHA_CORTE"), "CORTE AL DIA").otherwise("")
} 

curva_intentos_score = curva_intentos_score.withColumns(ajustes_cols1)

c = {'25-1': 'C1','25-2': 'C2','25-3': 'C3'}
curva_intentos_score = f_amr.map_cols(curva_intentos_score,'CICLO',c,'CICLO_LAUREATE')

# Dummies por etapa
etapas = ['LEADS', 'CITAS', 'ASISTENCIAS', 'INSCRITOS']
for i in etapas:
    curva_intentos_score = curva_intentos_score.withColumn(i, f_.when(f_.col('ETAPA') == i,f_.lit(1)).otherwise(f_.lit(0)) )

#Completitud de scores nulos ,
sin_score_1 = ['general_risk_bi', 'general_risk_crm', 'general_risk_prueba', 'general_risk_unicos']
for i in sin_score_1:
    curva_intentos_score = curva_intentos_score.withColumn(i, f_.when(f_.col(i).isNull(), "S/SCORE").otherwise(f_.col(i)))

sin_score_bin = [('S/SCORE_bi','general_risk_bi'),('S/SCORE_crm', 'general_risk_crm'),( 'S/SCORE_prueba','general_risk_prueba'),('S/SCORE_unicos','general_risk_unicos')]
for (i, j) in sin_score_bin:
    curva_intentos_score = curva_intentos_score.withColumn(i, f_.when(f_.col(j) == "S/SCORE",1).otherwise(0))

#Completitud de scores nulos
sin_score_2 = [('S/SCORE_LS','LEAD_SCORE'), ('S/SCORE_CS','CALL_SCORE'), ('S/SCORE_UNICOS_H','SCORE_UNICOS')]
for (i,j) in sin_score_2:
    curva_intentos_score = curva_intentos_score.withColumn(i, f_.when(f_.col(j).isNull(), 1).otherwise(0))

# Generación de intervalos
intervalos = [('INTERVALO_SC_bi', 'S/SCORE_LS', 'LEAD_SCORE'),
('INTERVALO_SC_crm', 'S/SCORE_CS', 'CALL_SCORE'),
('INTERVALO_SC_unicos', 'S/SCORE_UNICOS_H', 'SCORE_UNICOS')]
for (i,j,k) in intervalos:
    curva_intentos_score = curva_intentos_score.withColumn(i, f_.when(f_.col(j)==1, 'S/SCORE')\
                                                                .when(f_.col(k)>= 90, '[90,100]')\
                                                                .when(f_.col(k)>= 80, '[80,90)')\
                                                                .when(f_.col(k)>= 70, '[70,80)')\
                                                                .when(f_.col(k)>= 60, '[60,70)')\
                                                                .when(f_.col(k)>= 50, '[50,60)')\
                                                                .when(f_.col(k)>= 40, '[40,50)')\
                                                                .when(f_.col(k)>= 30, '[30,40)')\
                                                                .when(f_.col(k)>= 20, '[20,30)')\
                                                                .when(f_.col(k)>= 10, '[10,20)')\
                                                                .otherwise('[0,10)'))

#identificadores score
id_score = {"GREEN":'1',"YELLOW":'2',"ORANGE":'3',"RED":'4',"S/SCORE":"5"}
ids_score = [('general_risk_bi', 'ID_General_Risk_bi'),
('general_risk_crm', 'ID_General_Risk_crm'),
('general_risk_prueba','ID_General_Risk_prueba'),
('general_risk_unicos', 'ID_General_Risk_unicos')]
for (i,j) in ids_score:
    curva_intentos_score = f_amr.map_cols(curva_intentos_score, i , id_score, j)

id_rango = {"[90,100]":'10', "[80,90)":'9', "[70,80)":'8', "[60,70)":'7' ,"[50,60)":'6' ,"[40,50)":'5' ,"[30,40)":'4',"[20,30)":'3',"[10,20)":'2', "[0,10)":'1',"S/SCORE":'11'}
ids_rango = [('INTERVALO_SC_bi', 'ORDEN_INTERVALO_bi'),
('INTERVALO_SC_crm', 'ORDEN_INTERVALO_crm'),
('INTERVALO_SC_unicos','ORDEN_INTERVALO_unicos')]
for (i,j) in ids_rango:
    curva_intentos_score = f_amr.map_cols(curva_intentos_score, i , id_rango, j)

for i in ['BLN', 'CAMPUS_BI', 'FUENTE', 'FUENTE_AGRUPADA', 'LINEA_AGRUPADA', \
'LINEA_NEGOCIO_BI',	'MERCADO', 'MODALIDAD_BI', 'MODALIDAD_AGRUPADA', 'PROGRAMA_BI',	\
'RECICLADO', 'REGION_BI', 'TEAM_PRODUCTOR', 'TIPO_DE_INGRESO_BI', 'TIPO_INSCRIPCION', \
'TIPO_RECICLADO', 'TIPO_TRAMITE', 'VERTICAL_BI', 'ZONA_BI']:
    curva_intentos_score = curva_intentos_score.withColumn(i, f_.upper(f_.col(i)))


StatementMeta(, 08303616-ced7-4858-b382-9c890d31be05, 34, Finished, Available, Finished)

In [ ]:
from pyspark.sql.functions import current_timestamp
curva_intentos_score = curva_intentos_score.withColumn("ACTUALIZACION", f_.expr("current_timestamp() - INTERVAL 6 HOURS"))
p1= 'abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/5182b19a-e57c-4163-83a9-b0b90997f691/Files/CUBOS/CURVA_INTENTOS_UVM'
curva_intentos_score.repartition(10).write.mode("overwrite").parquet(p1)